In [1]:
import warnings
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR
from src import load_dataset as load
from src.QCNN_layers.Conv_layer import Conv_RBS_density_I2_3D
from src.QCNN_layers.Measurement_layer import measurement
from src.QCNN_layers.Pooling_layer import Pooling_3D_density
from src.training import train_globally
from src.QCNN_layers.Dense_layer import Dense_RBS_density_3D, Basis_Change_I_to_HW_density_3D, Trace_out_dimension
from src.list_gates import drip_circuit, full_pyramid_circuit

warnings.simplefilter('ignore')

In [2]:
##################### Hyperparameters begin #######################
# Below are the hyperparameters of this network, you can change them to test
I = 8  # dimension of image we use. If you use 2 times conv and pool layers, please make it a multiple of 4
O = I // 2  # dimension after pooling, usually you don't need to change this
J = 4  # number of channel
k = 3  # preserving subspace parameter, usually you don't need to change this
K = 4  # size of kernel in the convolution layer, please make it divisible by O=I/2
stride = 1 # the difference in step sizes for different channels
batch_size = 10  # batch number
training_dataset = 10  # training dataset sample number
testing_dataset = 10  # testing dataset sample number
is_shuffle = False # shuffle for this dataset
learning_rate = 1e-1 # step size for each learning steps
train_epochs = 10  # number of epoch we train
test_interval = 10  # when the training epoch reaches an integer multiple of the test_interval, print the testing result
criterion = torch.nn.CrossEntropyLoss() # loss function
device = torch.device("cpu")  # also torch.device("cpu"), or torch.device("mps") for macbook

# Here you can modify the RBS gate list that you want for the dense layer:
# dense_full_gates is for the case qubit=O+J, dense_reduce_gates is for the case qubit=5.
# Why we need two dense gate lists? Because for the 10 labels classification we only need 10 dimension in the end,
# so after the full dense we reduce the dimension from binom(O+J,3) to binom(5,3)=10, i.e., only keep the last 5 qubits.
# Finally, we do the reduce dense for 5 qubits and measurement.
# Also, you can check visualization of different gate lists in the file "src/list_gates.py"
dense_full_gates = drip_circuit(O + J)
dense_reduce_gates = full_pyramid_circuit(5)
##################### Hyperparameters end #######################

In [3]:
class QCNN(nn.Module):
    """
    Hamming weight preserving quantum convolution neural network (k=3)

    Tensor dataflow of this network:
    input density matrix: (batch,J*I^2,J*I^2)--> conv1: (batch,J*I^2,J*I^2)--> pool1: (batch,J*O^2,J*O^2)
    --> conv2: (batch,J*O^2,J*O^2)--> pool2: (batch,J*(O/2)^2,J*(O/2)^2)--> basis_map: (batch,binom(O+J,3),binom(O+J,3))
    --> full_dense: (batch,binom(O+J,3),binom(O+J,3)) --> reduce_dim: (batch,binom(5,3)=10,10)
    --> reduce_dense: (batch,10,10) --> output measurement: (batch,10)

    Then we can use it to calculate the Loss(output, targets)
    """
    def __init__(self, I, O, J, K, k, dense_full_gates, dense_reduce_gates, device):
        """ Args:
            - I: dimension of image we use, default I is 28
            - O: dimension of image we use after a single pooling
            - J: number of convolution channels
            - K: size of kernel
            - k: preserving subspace parameter, it should be 3
            - dense_full_gates: dense gate list, dimension from binom(O+J,3) to binom(5,3)=10
            - dense_reduce_gates: reduced dense gate list, dimension from 10 to 10
            - device: torch device (cpu, cuda, etc...)
        """
        super(QCNN, self).__init__()
        self.conv1 = Conv_RBS_density_I2_3D(I, K, J, device)
        self.pool1 = Pooling_3D_density(I, O, J, device)
        self.conv2 = Conv_RBS_density_I2_3D(O, K, J, device)
        self.pool2 = Pooling_3D_density(O, O // 2, J, device)
        self.basis_map = Basis_Change_I_to_HW_density_3D(O // 2, J, k, device)
        self.dense_full = Dense_RBS_density_3D(O // 2, J, k, dense_full_gates, device)
        self.reduce_dim = Trace_out_dimension(10, device)
        self.dense_reduced = Dense_RBS_density_3D(0, 5, k, dense_reduce_gates, device)

    def forward(self, x):
        x = self.pool1(self.conv1(x))  # first convolution and pooling
        x = self.pool2(self.conv2(x))  # second convolution and pooling
        x = self.basis_map(x)  # basis change from 3D Image to HW=3
        x = self.dense_reduced(self.reduce_dim(self.dense_full(x)))  # dense layer
        return measurement(x, device)  # measure, only keep the diagonal elements


network = QCNN(I, O, J, K, k, dense_full_gates, dense_reduce_gates, device)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Loading data
train_loader, test_loader = load.load_MNIST(batch_size=batch_size, shuffle=is_shuffle)
reduced_train_loader = load.reduce_MNIST_dataset(train_loader, training_dataset, is_train=True)
reduced_test_loader = load.reduce_MNIST_dataset(test_loader, testing_dataset, is_train=False)

# training part
network_state = train_globally(batch_size, I, J, network, reduced_train_loader, reduced_test_loader, optimizer, scheduler, criterion, train_epochs, test_interval, stride, device)

Start training! Number of network total parameters: 68
Evaluation on test set: Loss = 2.302510, accuracy = 10.0000 %
Epoch 0: Loss = 2.348485, accuracy = 10.0000 %, time=1.7233s
Epoch 1: Loss = 2.258430, accuracy = 20.0000 %, time=1.6374s
Epoch 2: Loss = 2.210091, accuracy = 30.0000 %, time=1.6457s
Epoch 3: Loss = 2.121131, accuracy = 30.0000 %, time=1.8386s
Epoch 4: Loss = 2.057168, accuracy = 20.0000 %, time=1.6840s
Epoch 5: Loss = 2.024314, accuracy = 30.0000 %, time=1.6306s
Epoch 6: Loss = 2.013908, accuracy = 30.0000 %, time=1.6249s
Epoch 7: Loss = 1.994074, accuracy = 30.0000 %, time=1.6241s
Epoch 8: Loss = 1.967503, accuracy = 30.0000 %, time=1.6221s
Epoch 9: Loss = 1.948718, accuracy = 30.0000 %, time=1.6238s
Evaluation on test set: Loss = 2.234458, accuracy = 20.0000 %
